In [76]:
import pandas as pd

df = pd.read_excel("ny_citibikes_raw.xlsx")
df.head()

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,2017-01-01 00:38:00,2017-01-01 01:03:00,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
3,2017-01-01 01:56:00,2017-01-01 02:00:00,3186,Grove St PATH,3270,Jersey & 6th St,24604,Subscriber,1970,51,45-54,258,4,1,Winter,10,Sunday
4,2017-01-01 02:12:00,2017-01-01 02:23:00,3270,Jersey & 6th St,3206,Hilltop,24641,Subscriber,1978,43,35-44,663,11,1,Winter,10,Sunday


1. Dataset Exploration
  - What information does each column contain?


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20400 entries, 0 to 20399
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Start Time            20400 non-null  datetime64[ns]
 1   Stop Time             20400 non-null  datetime64[ns]
 2   Start Station ID      20400 non-null  int64         
 3   Start Station Name    20400 non-null  object        
 4   End Station ID        20400 non-null  int64         
 5   End Station Name      20399 non-null  object        
 6   Bike ID               20400 non-null  int64         
 7   User Type             20400 non-null  object        
 8   Birth Year            20400 non-null  int64         
 9   Age                   20400 non-null  int64         
 10  Age Groups            20400 non-null  object        
 11  Trip Duration         20400 non-null  int64         
 12  Trip_Duration_in_min  20400 non-null  int64         
 13  Month           

  - Are there missing or duplicated values?
  

In [78]:
missing = df.count()
missing
print(f"Total de valores faltantes:\n{missing}")

Total de valores faltantes:
Start Time              20400
Stop Time               20400
Start Station ID        20400
Start Station Name      20400
End Station ID          20400
End Station Name        20399
Bike ID                 20400
User Type               20400
Birth Year              20400
Age                     20400
Age Groups              20400
Trip Duration           20400
Trip_Duration_in_min    20400
Month                   20400
Season                  20400
Temperature             20400
Weekday                 20400
dtype: int64


In [79]:
nulo = df[df["End Station Name"].isnull()]
nulo

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
9858,2017-02-25 12:36:00,2017-02-25 12:44:00,3220,5 Corners Library,3211,NaN,24522,Subscriber,1963,58,55-64,532,9,2,Winter,11,Saturday


In [80]:
duplicated = df.duplicated()
count_duplicated = duplicated[duplicated == True].count()
print("Total de valores duplicados:",count_duplicated)

Total de valores duplicados: 3555


In [81]:
df = df.drop_duplicates()

- What is the overall time span of the trips?

In [82]:
tiempo_total_uso = df["Trip Duration"].sum()
tiempo_total_uso_horas = (tiempo_total_uso/360).round(2)
print(f"Horas de servicio durante 2017:", tiempo_total_uso)

Horas de servicio durante 2017: 9680893


2. Basic Statistics
  - What is the average trip duration (in minutes)?

In [83]:
trip_min_mean = df["Trip_Duration_in_min"].mean().round(2)
print(f"La duración media de los viajes en 2025 ha sido de: {trip_min_mean} minutos")

La duración media de los viajes en 2025 ha sido de: 9.58 minutos


- What is the minimum and maximum duration?

In [ ]:
#Min Values
viaje_mas_corto = df["Trip Duration"].min()
a = df.loc[df["Trip Duration"] == viaje_mas_corto]
print(a)
mes_viajes_mas_cortos=0
#print(f"El viaje más corto tuvo una duración de {viaje_mas_corto} segundos")
#Max Values
viaje_mas_largo = 0
mes_viajes_mas_largos = 0



               Start Time           Stop Time  Start Station ID  \
1787  2017-01-13 07:47:00 2017-01-13 07:48:00              3267   
3865  2017-01-25 16:23:00 2017-01-25 16:24:00              3211   
8471  2017-02-20 13:25:00 2017-02-20 13:26:00              3270   
12553 2017-03-05 08:26:00 2017-03-05 08:27:00              3270   

      Start Station Name  End Station ID End Station Name  Bike ID  \
1787        Morris Canal            3267     Morris Canal    26188   
3865          Newark Ave            3211       Newark Ave    26217   
8471     Jersey & 6th St            3272     Jersey & 3rd    26192   
12553    Jersey & 6th St            3272     Jersey & 3rd    26259   

        User Type  Birth Year  Age Age Groups  Trip Duration  \
1787   Subscriber        1980   41      35-44             61   
3865   Subscriber        1976   45      45-54             61   
8471   Subscriber        1978   43      35-44             61   
12553  Subscriber        1981   40      35-44            

In [100]:
horas = df["Start Time"].dt.month
horas

0        1
1        1
3        1
4        1
5        1
        ..
20393    3
20394    3
20396    3
20398    3
20399    3
Name: Start Time, Length: 16845, dtype: int32



  
  - What are the most common start and end stations?






3. Users and Demographics
  - How many unique bikes were used?
  - What are the proportions of user types (Subscriber vs Customer)?
  - What is the age distribution of the users? Which age group uses the service the most?


4. Temporal Analysis
  - How does the number of trips vary by weekday?
  - Which month or season has the most rides?
  - What time of day do most trips start?


5. Geographic Analysis
  - Which station pairs (start → end) appear most often?
  - Are there any stations that appear only as start or only as end stations?


6. Temperature and Duration
  - Is there any visible relationship between temperature and trip duration?

  - How does average trip duration vary by season?


7. Summary and Interpretation
  - Write a short summary (5–10 lines) of your findings.
  - Mention patterns, anomalies, or interesting trends you observed.

Instructions

Use dataset from "Xeral/Datasets" on this page course
Create a github repo for this task.
Clone your repo locally.
Download the dataset (don't forget to commit)
Create a new environment to run the notebook.
Create a .ipynb notebook with clear structure and clean outputs.
Markdown answers after each question.
At least one meaningful chart (e.g., trips by weekday, average duration by season).
Upload your code (don't forget a nice Readme.md).
Paste the github repository link below (don't forget inviting me if it's a private repo).